In [1]:
import torch.nn as nn
from src import (train_sdf, get_sdf_data_loader, deep_mind_loss, plot_sdf_results, plot_sdf_results_over_line,
                 EncodeProcessDecode, EncodeProcessDecodeNEW, GraphNetworkIndependentBlock, GraphNetworkBlock)

In [2]:
# get data
n_objects = 100
data_folder = "C:/Users/amaleki/OneDrive - ANSYS, Inc/Projects/mesh_gen/mesh_sdf/mesh/"
batch_size = 6
edge_method = 'edge' # or 'proximity'
edge_params = {'radius': 0.05}
train_data, test_data = get_sdf_data_loader(n_objects, data_folder, batch_size, eval_frac=0.2,
                                            edge_method=edge_method, edge_params=edge_params)

preparing sdf data loader


In [3]:
n_edge_feat_in, n_edge_feat_out = 5, 1
n_node_feat_in, n_node_feat_out = 3, 1
n_global_feat_in, n_global_feat_out = 3, 3
mlp_latent_size = 32
num_processing_steps = 10
model = EncodeProcessDecodeNEW(n_edge_feat_in=n_edge_feat_in, n_edge_feat_out=n_edge_feat_out,
                               n_node_feat_in=n_node_feat_in, n_node_feat_out=n_node_feat_out,
                               n_global_feat_in=n_global_feat_in, n_global_feat_out=n_global_feat_out,
                               mlp_latent_size=mlp_latent_size, num_processing_steps=num_processing_steps,
                               encoder=GraphNetworkIndependentBlock, decoder=GraphNetworkIndependentBlock,
                               processor=GraphNetworkBlock, output_transformer=GraphNetworkIndependentBlock,
                               full_output=True
                               )

In [4]:
loss_funcs = [deep_mind_loss]

In [6]:
# train
gamma       = 0.5
lr_0        = 0.001
n_epoch     = 500
step_size   = 250
print_every = 25
save_name   = "deep_mind_graph_" + edge_method + "_nn_" + str(mlp_latent_size) + "_nlayers_" + str(num_processing_steps)
train_sdf(model, train_data, train_data, loss_funcs, n_epoch=n_epoch, print_every=print_every, use_cpu=True,
          save_name=save_name, lr_0=lr_0, step_size=step_size, gamma=gamma)

KeyboardInterrupt: 

In [ ]:
# visualization
data_loader, _ = get_sdf_data_loader(10, data_folder, 1, eval_frac=0, edge_method=edge_method, edge_params=edge_params)
output_func = lambda x: x[-1][1]
plot_sdf_results(model, data_loader, save_name=save_name, get_output=output_func, levels=[-0.1, 0., 0.1])